In [1]:
import os
os.environ["SPARK_HOME"] = "/home/hadoop/spark"

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName = "tomato_score")

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

24/06/13 09:37:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, to_date, udf
from pyspark.sql.types import DoubleType
import os
import numpy as np

# 새로운 SparkSession 생성
spark = SparkSession.builder \
    .appName("Tomato Data Analysis Score") \
    .getOrCreate()

# 파일 경로 설정
file_path = "hdfs://master01:9000/user/hadoop/tomato_avg.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
pdf=df.toPandas()

# 각 열의 평균과 표준 편차 계산
stats = pdf.describe().transpose()
means = stats['mean'].to_dict()
std_devs = stats['std'].to_dict()
means_bc = spark.sparkContext.broadcast(means)
std_devs_bc = spark.sparkContext.broadcast(std_devs)

# 가우시안 분포 계산
def gaussian(x, col):
    mean = means_bc.value[col]
    std_dev = std_devs_bc.value[col]
    return (1 / (std_dev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean) / std_dev) ** 2)

gaussian_udf=udf(lambda x, col: gaussian(x, col), DoubleType())
print(gaussian_udf(df['PFBS_NTRO_CBDX_CTRN'], 'PFBS_NTRO_CBDX_CTRN'))

24/06/13 09:37:28 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


Column<b'<lambda>(PFBS_NTRO_CBDX_CTRN, PFBS_NTRO_CBDX_CTRN)'>


In [33]:
file_path = "hdfs://master01:9000/user/hadoop/tomato_avg.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
pdf = df.toPandas()

# 각 열의 평균과 표준 편차 계산
stats = pdf.describe().transpose()
means = stats['mean']
std_devs = stats['std']

save_path="/home/hadoop/tavg.csv"
pd.DataFrame({'means': means, 'std_devs': std_devs}).to_csv(save_path)

In [37]:
p=pd.read_csv('/home/hadoop/tomato_avg.csv')
stats = p.describe().transpose()
means = stats['mean']
std_devs = stats['std']
print(means)

ABSLT_HMDT         12.975628
INNER_TPRT_1       18.778564
FRST_CLUSTER       11.893882
CLR_OPRT_YN_2       0.000000
SPRYN_DEVICE        0.019468
                     ...    
TRWVLV_OPMD_1       0.226009
FMGEQ_OPRT_YN       0.129801
INNER_HMDT_1       79.520117
NTSLT_SPL_ELCDT     2.932592
CBDX_GNRT_OPMD      0.690801
Name: mean, Length: 88, dtype: float64


In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType
import pandas as pd
import numpy as np

# 새로운 SparkSession 생성
spark = SparkSession.builder \
    .appName("Tomato Data Analysis Score") \
    .getOrCreate()

# 파일 경로 설정
file_path = "hdfs://master01:9000/user/hadoop/tomato_avg.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
pdf = df.toPandas()

# 각 열의 평균과 표준 편차 계산
stats = pdf.describe().transpose()
means = stats['mean']
std_devs = stats['std']

# 가우시안 분포 계산 함수
def gaussian(x, mean, std_dev):
    return (1 / (std_dev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean) / std_dev) ** 2)

# 각 컬럼에 대한 가우시안 값을 계산하고 가중치 적용
pdf['Score'] = (
    gaussian(pdf['PFBS_NTRO_CBDX_CTRN'], means['PFBS_NTRO_CBDX_CTRN'], std_devs['PFBS_NTRO_CBDX_CTRN']) * 560 +
    gaussian(pdf['EXTN_TPRT'], means['EXTN_TPRT'], std_devs['EXTN_TPRT']) * 42 +
    gaussian(pdf['STRTN_WATER'], means['STRTN_WATER'], std_devs['STRTN_WATER']) * 12 +
    gaussian(pdf['WATER_LACK_VL'], means['WATER_LACK_VL'], std_devs['WATER_LACK_VL']) * 30 +
    gaussian(pdf['EXTN_ACCMLT_QOFLG'], means['EXTN_ACCMLT_QOFLG'], std_devs['EXTN_ACCMLT_QOFLG']) * 1700 * 8 +
    gaussian(pdf['NTSLT_SPL_PH_LVL'], means['NTSLT_SPL_PH_LVL'], std_devs['NTSLT_SPL_PH_LVL']) * 5.5 +
    gaussian(pdf['NTSLT_SPL_ELCDT'], means['NTSLT_SPL_ELCDT'], std_devs['NTSLT_SPL_ELCDT']) * 8.5 +
    gaussian(pdf['AVE_INNER_TPRT_1_2'], means['AVE_INNER_TPRT_1_2'], std_devs['AVE_INNER_TPRT_1_2']) * 72 +
    gaussian(pdf['AVE_INNER_HMDT_1_2'], means['AVE_INNER_HMDT_1_2'], std_devs['AVE_INNER_HMDT_1_2']) * 168
)

save_path="/home/hadoop/tscore.csv"
pdf.to_csv(save_path)

# 명시적 스키마 정의
schema = StructType([
    StructField("PFBS_NTRO_CBDX_CTRN", DoubleType(), True),
    StructField("EXTN_TPRT", DoubleType(), True),
    StructField("STRTN_WATER", DoubleType(), True),
    StructField("WATER_LACK_VL", DoubleType(), True),
    StructField("EXTN_ACCMLT_QOFLG", DoubleType(), True),
    StructField("NTSLT_SPL_PH_LVL", DoubleType(), True),
    StructField("NTSLT_SPL_ELCDT", DoubleType(), True),
    StructField("AVE_INNER_TPRT_1_2", DoubleType(), True),
    StructField("AVE_INNER_HMDT_1_2", DoubleType(), True),
    StructField("Score", DoubleType(), True)
])

# Pandas DataFrame을 다시 Spark DataFrame으로 변환
#result_df = spark.createDataFrame(pdf)

# 결과 확인
#result_df.show()


In [40]:
file_path = "hdfs://master01:9000/user/hadoop/strawberry_avg.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
pdf = df.toPandas()

# 각 열의 평균과 표준 편차 계산
stats = pdf.describe().transpose()
means = stats['mean']
std_devs = stats['std']

# 가우시안 분포 계산 함수
def gaussian(x, mean, std_dev):
    return (1 / (std_dev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean) / std_dev) ** 2)

# 각 컬럼에 대한 가우시안 값을 계산하고 가중치 적용
pdf['Score'] = (
    gaussian(pdf['PFBS_NTRO_CBDX_CTRN'], means['PFBS_NTRO_CBDX_CTRN'], std_devs['PFBS_NTRO_CBDX_CTRN']) * 750 +
    gaussian(pdf['EXTN_TPRT'], means['EXTN_TPRT'], std_devs['EXTN_TPRT']) * 144 +
    gaussian(pdf['STRTN_WATER'], means['STRTN_WATER'], std_devs['STRTN_WATER']) * 144 +
    gaussian(pdf['WATER_LACK_VL'], means['WATER_LACK_VL'], std_devs['WATER_LACK_VL']) * 117+
    gaussian(pdf['EXTN_ACCMLT_QOFLG'], means['EXTN_ACCMLT_QOFLG'], std_devs['EXTN_ACCMLT_QOFLG']) * 2100 +
    gaussian(pdf['AVE_INNER_TPRT_1_2'], (means['AVE_INNER_TPRT_1_2']+means['AVE_INNER_TPRT_3_4'])/2, (std_devs['AVE_INNER_TPRT_1_2']+std_devs['AVE_INNER_TPRT_1_2'])/2) * 21*12.5 +
    gaussian(pdf['AVE_INNER_HMDT_1_2'], (means['AVE_INNER_HMDT_1_2']+means['AVE_INNER_HMDT_3_4'])/2, (std_devs['AVE_INNER_HMDT_1_2']+std_devs['AVE_INNER_HMDT_3_4'])/2) * 360
)

save_path="/home/hadoop/sscore.csv"
pdf.to_csv(save_path)


/home/hadoop/spark/python/pyspark/sql/dataframe.py:2195: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


In [ ]:
spark = SparkSession.builder \
    .appName("Strawberry Data Analysis Score") \
    .getOrCreate()

# 파일 경로 설정
file_path = "hdfs://master01:9000/user/hadoop/strawberry_avg.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
pdf = df.toPandas()

# 각 열의 평균과 표준 편차 계산
stats = pdf.describe().transpose()
means = stats['mean']
std_devs = stats['std']

# 각 컬럼에 대한 가우시안 값을 계산하고 가중치 적용
pdf['Score'] = (
    gaussian(pdf['PFBS_NTRO_CBDX_CTRN'], means['PFBS_NTRO_CBDX_CTRN'], std_devs['PFBS_NTRO_CBDX_CTRN']) * 560 +
    gaussian(pdf['EXTN_TPRT'], means['EXTN_TPRT'], std_devs['EXTN_TPRT']) * 42 +
    gaussian(pdf['STRTN_WATER'], means['STRTN_WATER'], std_devs['STRTN_WATER']) * 12 +
    gaussian(pdf['WATER_LACK_VL'], means['WATER_LACK_VL'], std_devs['WATER_LACK_VL']) * 30 +
    gaussian(pdf['EXTN_ACCMLT_QOFLG'], means['EXTN_ACCMLT_QOFLG'], std_devs['EXTN_ACCMLT_QOFLG']) * 1700 * 8 +
    gaussian(pdf['NTSLT_SPL_PH_LVL'], means['NTSLT_SPL_PH_LVL'], std_devs['NTSLT_SPL_PH_LVL']) * 5.5 +
    gaussian(pdf['NTSLT_SPL_ELCDT'], means['NTSLT_SPL_ELCDT'], std_devs['NTSLT_SPL_ELCDT']) * 8.5 +
    gaussian(pdf['AVE_INNER_TPRT_1_2'], means['AVE_INNER_TPRT_1_2'], std_devs['AVE_INNER_TPRT_1_2']) * 72 +
    gaussian(pdf['AVE_INNER_HMDT_1_2'], means['AVE_INNER_HMDT_1_2'], std_devs['AVE_INNER_HMDT_1_2']) * 168
)

save_path="/home/hadoop/sscore.csv"
pdf.to_csv(save_path)

In [19]:
import pandas as pd
import numpy as np

# 새로운 SparkSession 생성
spark = SparkSession.builder \
    .appName("Tomato Data Analysis Score") \
    .getOrCreate()

# 파일 경로 설정
file_path = "hdfs://master01:9000/user/hadoop/tomato_avg.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
pdf = df.toPandas()

# 각 열의 평균과 표준 편차 계산
stats = pdf.describe().transpose()
means = stats['mean']
std_devs = stats['std']

# 가우시안 분포 계산 함수
def gaussian(x, mean, std_dev):
    return (1 / (std_dev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean) / std_dev) ** 2)

# 각 컬럼에 대한 가우시안 값을 계산하고 가중치 적용
pdf['Score'] = (
    gaussian(pdf['PFBS_NTRO_CBDX_CTRN'], means['PFBS_NTRO_CBDX_CTRN'], std_devs['PFBS_NTRO_CBDX_CTRN']) * 560 +
    gaussian(pdf['EXTN_TPRT'], means['EXTN_TPRT'], std_devs['EXTN_TPRT']) * 42 +
    gaussian(pdf['STRTN_WATER'], means['STRTN_WATER'], std_devs['STRTN_WATER']) * 12 +
    gaussian(pdf['WATER_LACK_VL'], means['WATER_LACK_VL'], std_devs['WATER_LACK_VL']) * 30 +
    gaussian(pdf['EXTN_ACCMLT_QOFLG'], means['EXTN_ACCMLT_QOFLG'], std_devs['EXTN_ACCMLT_QOFLG']) * 1700 * 8 +
    gaussian(pdf['NTSLT_SPL_PH_LVL'], means['NTSLT_SPL_PH_LVL'], std_devs['NTSLT_SPL_PH_LVL']) * 5.5 +
    gaussian(pdf['NTSLT_SPL_ELCDT'], means['NTSLT_SPL_ELCDT'], std_devs['NTSLT_SPL_ELCDT']) * 8.5 +
    gaussian(pdf['AVE_INNER_TPRT_1_2'], means['AVE_INNER_TPRT_1_2'], std_devs['AVE_INNER_TPRT_1_2']) * 72 +
    gaussian(pdf['AVE_INNER_HMDT_1_2'], means['AVE_INNER_HMDT_1_2'], std_devs['AVE_INNER_HMDT_1_2']) * 168
)

# 결과 확인
pdf.head()

,date,ABSLT_HMDT,INNER_TPRT_1,FRST_CLUSTER,CLR_OPRT_YN_2,SPRYN_DEVICE,FRST_TREE_CNT,TRWVLV_OPDR_RATE_2,SKLT_OPMD_2_LEFT,SKLT_OPMD_1_LEFT,...,RTTN_PUMP_OPRT_YN_2,PLT_LNGTH,CLR_OPRT_YN_3,HRZNT_SCRN_OPDR_RATE_2,TRWVLV_OPMD_1,FMGEQ_OPRT_YN,INNER_HMDT_1,NTSLT_SPL_ELCDT,CBDX_GNRT_OPMD,Score
0,2022-10-07,13.789697,21.755455,NaN,0.0,0.060606,NaN,100.000000,0.0,1.000000,...,0.0,1061.92,0.0,99.533333,0.0,0.0,70.380606,3.363636,0.0,36.459905
1,2022-10-08,12.718576,20.255208,NaN,0.0,0.059028,NaN,77.430556,0.0,1.000000,...,0.0,1061.92,0.0,99.916667,0.0,0.0,71.888368,3.258333,0.0,50.744532
2,2022-10-09,14.920139,19.863194,NaN,0.0,0.000000,NaN,65.460069,0.0,0.989583,...,0.0,1061.92,0.0,100.000000,0.0,0.0,85.157813,3.279167,0.0,53.207717
3,2022-10-10,12.748437,19.002257,NaN,0.0,0.138889,NaN,56.482639,0.0,1.000000,...,0.0,1061.92,0.0,100.000000,0.0,0.0,77.963715,3.050000,0.0,57.161420
4,2022-10-11,12.304167,19.358507,NaN,0.0,0.083333,NaN,61.159722,0.0,1.000000,...,0.0,1061.92,0.0,100.000000,0.0,0.0,73.693576,2.970833,0.0,57.774875


In [12]:
df = df.withColumn('Score', 
    gaussian_udf(col('PFBS_NTRO_CBDX_CTRN'), F.lit('PFBS_NTRO_CBDX_CTRN')) * 560 +
    gaussian_udf(col('EXTN_TPRT'), F.lit('EXTN_TPRT')) * 42 +
    gaussian_udf(col('STRTN_WATER'), F.lit('STRTN_WATER')) * 12 +
    gaussian_udf(col('WATER_LACK_VL'), F.lit('WATER_LACK_VL')) * 30 +
    gaussian_udf(col('EXTN_ACCMLT_QOFLG'), F.lit('EXTN_ACCMLT_QOFLG')) * 1700 * 8 +
    gaussian_udf(col('NTSLT_SPL_PH_LVL'), F.lit('NTSLT_SPL_PH_LVL')) * 5.5 +
    gaussian_udf(col('NTSLT_SPL_ELCDT'), F.lit('NTSLT_SPL_ELCDT')) * 8.5 +
    gaussian_udf(col('AVE_INNER_TPRT_1_2'), F.lit('AVE_INNER_TPRT_1_2')) * 72 +
    gaussian_udf(col('AVE_INNER_HMDT_1_2'), F.lit('AVE_INNER_HMDT_1_2')) * 168
)

In [13]:
df.show()

24/06/11 08:41:20 ERROR Executor: Exception in task 0.0 in stage 18.0 (TID 18)
net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$anonfun$evaluate$1.apply(BatchEvalPythonExec.scala:94)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$anonfun$evaluate$1.apply(BatchEvalPythonExec.scala:93)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:441)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator

Py4JJavaError: An error occurred while calling o565.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 1 times, most recent failure: Lost task 0.0 in stage 18.0 (TID 18, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$anonfun$evaluate$1.apply(BatchEvalPythonExec.scala:94)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$anonfun$evaluate$1.apply(BatchEvalPythonExec.scala:93)
	at scala.collection.Iterator$$anon$12.nextCur(Itera

In [54]:
df=df.withColumn('Score', gaussian_udf(col('PFBS_NTRO_CBDX_CTRN'), 'PFBS_NTRO_CBDX_CTRN')*560+gaussian_udf(col('EXTN_TPRT'),'EXTN_TPRT')*42+ \
                 gaussian_udf(col('STRTN_WATER'), 'STRTN_WATER')*12+gaussian_udf(col('WATER_LACK_VL'), 'WATER_LACK_VL')*30+ \
                 gaussian_udf(col('EXTN_ACCMLT_QOFLG'), 'EXTN_ACCMLT_QOFLG')*1700*8+ gaussian_udf(col('NTSLT_SPL_PH_LVL'), 'NTSLT_SPL_PH_LVL')*5.5+ \
                 gaussian_udf(col('NTSLT_SPL_ELCDT'),'NTSLT_SPL_ELCDT')*8.5+gaussian_udf(col('AVE_INNER_TPRT_1_2'),'AVE_INNER_TPRT_1_2')*72+ \
                 gaussian_udf(col('AVE_INNER_HMDT_1_2'), 'AVE_INNER_HMDT_1_2')*168)

In [41]:
output_path = "/home/hadoop"
output_file = os.path.join(output_path, "tomato_score.csv")
df.coalesce(1).write.csv(output_file, header=True, mode='overwrite')

[Stage 62:>                                                         (0 + 1) / 1]24/06/11 06:56:18 ERROR Utils: Aborting task
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/hadoop/venv3.7/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 335.2151515151515

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/hadoop/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/

Py4JJavaError: An error occurred while calling o1047.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:202)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:136)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:132)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:160)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sc